In [ ]:
import cv2
import imageio
import time
from matplotlib import pyplot as plt
from PIL import Image  
from PIL import Image as im
import numpy as np
from PCV.localdescriptors import harris
from PCV.tools.imtools import imresize

from skimage.metrics import structural_similarity as ssim
import argparse
import shutil
import os


base_images=[]
folders=[]
folders_full=[]
save_folders=[]
csv_saves=[]
folder_4_base=r"E:\selected dataset\bases for all category" #folder containing base images of each target class
folder=r"E:\selected dataset\MIT withoutbase" #folder containing multiple classes folder of target images

save_folder=r"E:\selected dataset\demo save" 


for filename in os.listdir(folder_4_base):
        base_images.append((os.path.join(folder_4_base,filename)))
        
for filename in os.listdir(folder):
        folders.append((os.path.join(folder,filename)))
        
for filename in os.listdir(save_folder):
        save_folders.append(os.path.join(save_folder,filename))

M=12 #total number of boxes
N=25 #picked number of boxes
    

for main_loop in range(10): #number of classes 

    img2_arr=[]
    img3_arr=[]
    imgO_arr=[]
    imgO_arr_Harris=[]
    imgnew_arr=[]
    savee=[]
    for i in range(len(os.listdir(folders[main_loop]))):
        savee.append('\p'+str(i)+'.jpg')

    img1_arr=imageio.imread(base_images[main_loop])
    img1_arr_Harris=np.array(Image.open(base_images[main_loop]).convert("L"))
    a=0
    b=0

    for filename in os.listdir(folders[main_loop]):
            img2_arr.append(imageio.imread(os.path.join(folders[main_loop],filename)))
            imgO_arr.append(imageio.imread(os.path.join(folders[main_loop],filename)))
            imgO_arr_Harris.append(np.array(Image.open(os.path.join(folders[main_loop],filename)).convert("L")))
    v=0
 ########################################################################################################
    for i in range(len(img2_arr)):
        
        im_average=img2_arr[i]
        split1=[]
        split2=[]
        splitH=[]
        averaged=[]
        
        #decide box size according to your image size and M
        windowsize_r = 80
        windowsize_c = 80

        a=0
        for r in range(0,img1_arr.shape[0], windowsize_r):
            for c in range(0,img1_arr.shape[1], windowsize_c):
                arr=img1_arr[r:r+windowsize_r,c:c+windowsize_c,:]     
                image=im.fromarray(arr)
                split1.append(np.array(image))
        for r in range(0,img1_arr_Harris.shape[0], windowsize_r):
            for c in range(0,img1_arr_Harris.shape[1], windowsize_c):
                arr=img1_arr_Harris[r:r+windowsize_r,c:c+windowsize_c]     
                image=im.fromarray(arr)
                splitH.append(np.array(image))
        for r in range(0,img2_arr[i].shape[0], windowsize_r):
            for c in range(0,img2_arr[i].shape[1], windowsize_c):
                arr=img2_arr[i][r:r+windowsize_r,c:c+windowsize_c,:]     
                image=im.fromarray(arr)
                split2.append(np.array(image))
        im_average=split2.copy()


        saved=[]
        image_init_selected=[]
        image_target_selected=[]
        m1=[]
        maxed=[]
        for j in range(N):
            wid = 5
            harrisim = harris.compute_harris_response(splitH[j], 5)
            pts = harris.get_harris_points(harrisim, wid+1)
            pts=np.array(pts)
            m1.append(len(pts))
        
        for i in range(M):
            maxed.append(int(max(m1)))
            saved.append(m1.index(max(m1)))
            n1=m1.index(max(m1))
            m1[n1]=0
        for i in saved:
            image_init_selected.append(split1[i])
            image_target_selected.append(split2[i])


        #selected Average!
        def image_average(img1,img2):
                sum=np.average((img1, img2),axis=0)
                sum=sum.astype(np.uint8)
                return sum

        averaged=[]
        for j in range(len(image_target_selected)):
            image_target1=image_target_selected[j]
            image_init1=image_init_selected[j]
            averaged.append(image_average(image_target1,image_init1))
            
        l=0

        for i in saved:
                im_average[i]=averaged[l]
                l=l+1
        for i in range(N):
                arr2im=im.fromarray(im_average[i])
                arr2im.save('average'+str(i)+'.jpg')
        image1=[]
        blank_image = Image.new("RGB", (400, 400))
        for a in range(N):
            image1.append(Image.open('average'+str(a)+'.jpg'))


        list1=[0,80,160,240,320]
        k=0
        for i in list1:
            for j in list1:
                blank_image.paste(image1[k], (j,i))
                k=k+1
        plt.imshow(blank_image)
        plt.show()



  ##########################################################################################################          
        im2=blank_image
        im3=blank_image
      
        im2 = im2.save(save_folders[main_loop]+savee[v]) 
        imgnew_arr.append(imageio.imread(save_folders[main_loop]+savee[v]))
        v=v+1
    def calculate_SSIM(imageA,imageB):
        s = ssim(imageA, imageB,multichannel=True)
        
        return s
    SSIM1=[]
    for i in range(len(os.listdir(folders[main_loop]))):
        SSIM1.append(calculate_SSIM((imgO_arr[i]),(imgnew_arr[i])))
    print("SSIM between target & adversary image",SSIM1)
    def calculate_PSNR(im1,im2):
        P=cv2.PSNR(im1,im2)
        
        return P
    PSNR1=[]
    for i in range(len(os.listdir(folders[main_loop]))):
        PSNR1.append(calculate_PSNR(imgO_arr[i],imgnew_arr[i]))
    print("PSNR between target & adversary image",PSNR1)


    def feature_match(check):
        plt.imshow(adv_image/255)
        plt.show()
        plt.imshow(check/255)
        plt.show()
        wid = 5
        harrisim = harris.compute_harris_response(check, 5)
        filtered_coords1 = harris.get_harris_points(harrisim, wid+1)
        d1 = harris.get_descriptors(check, filtered_coords1, wid)

        harrisim = harris.compute_harris_response(adv_image, 5)
        filtered_coords2 = harris.get_harris_points(harrisim, wid+1)
        d2 = harris.get_descriptors(adv_image, filtered_coords2, wid)
        d1=np.array(d1)
        d2=np.array(d2)
        bf = cv2.BFMatcher()
        matches = bf.knnMatch(d1,d2, k=2)



        # Apply ratio test
        good = []
        for m,n in matches:
            if m.distance < 0.7*n.distance:
                good.append(m)
        filtered_coords1=np.array(filtered_coords1)
        filtered_coords2=np.array(filtered_coords2)



        src_pts1 = np.float32([ filtered_coords1[m.queryIdx] for m in good ]).reshape(-1,1,2)
        dst_pts1 = np.float32([ filtered_coords2[m.trainIdx] for m in good]).reshape(-1,1,2)
        try:
            M, mask = cv2.findHomography(src_pts1, dst_pts1, cv2.RANSAC,3.0)
            mask=list(mask)
            inliers=mask.count(1)
        except:
            inliers=0
       


        return inliers
    adv_image=img1_arr_Harris
    inliers_OP=[]
    for i in range(len(os.listdir(folders[main_loop]))):
            im2 = np.array(Image.open((save_folders[main_loop]+savee[i])).convert("L"))
            inliers_OP.append(feature_match(im2))
    adv_image=img1_arr_Harris
    inliers_OO=[]
    for i in range(len(os.listdir(folders[main_loop]))):
        inliers_OO.append(feature_match(imgO_arr_Harris[i]))
    print("feature match before attack",inliers_OO)
    print("feature match after attack",inliers_OP)
    PFMI=[]
    for i in range(len(os.listdir(folders[main_loop]))):
        try:
            PFMI.append(str(((inliers_OP[i]-inliers_OO[i])/inliers_OO[i])*100)+str('%'))
        except:
            PFMI.append(inliers_OP[i])
    print("percentage feature matching increase",PFMI)